In [1]:
from langchain_community.tools import TavilySearchResults
from langchain_groq import ChatGroq

from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langgraph.prebuilt import ToolNode

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [4]:
search_tool = TavilySearchResults(max_results=3)

tools = [search_tool]

/tmp/ipykernel_42442/1163154442.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=3)


In [5]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

llm_with_tools = llm.bind_tools(tools=tools)

In [6]:
def call_llm(state: AgentState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

In [7]:
def tools_router(state: AgentState):
    last_message = state["messages"][-1]
    
    if getattr(last_message, "tool_calls", None):
        return "tool_node"
    return END

In [8]:
tool_node = ToolNode(tools=tools)

In [9]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("call_llm", call_llm)
graph_builder.add_node("tool_node", tool_node)
graph_builder.set_entry_point("call_llm")

graph_builder.add_conditional_edges(
    "call_llm",
    tools_router,
)
graph_builder.add_edge("tool_node", "call_llm")

graph = graph_builder.compile()

In [12]:
input = {
    "messages": ["What's the current weather in Egypt?"]
}

events = graph.stream(input=input, stream_mode="values")

for event in events:
    print(event)

{'messages': [HumanMessage(content="What's the current weather in Egypt?", additional_kwargs={}, response_metadata={}, id='36b3e1d7-f6ab-4ef6-8564-c7a131437e95')]}
{'messages': [HumanMessage(content="What's the current weather in Egypt?", additional_kwargs={}, response_metadata={}, id='36b3e1d7-f6ab-4ef6-8564-c7a131437e95'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'mj08vmpta', 'function': {'arguments': '{"query":"current weather in Egypt"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 286, 'total_tokens': 307, 'completion_time': 0.052980376, 'prompt_time': 0.023716679, 'queue_time': 0.092975182, 'total_time': 0.076697055}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--042844cf-affb-4c58-94d5-1ce424095525-0', tool_calls=[{'name': 'tavily_search_result

In [13]:
input = {
    "messages": ["What's the current weather in Egypt?"]
}

events = graph.stream(input=input, stream_mode="updates")

for event in events:
    print(event)

{'call_llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'm1nn5vt3a', 'function': {'arguments': '{"query":"current weather in Egypt"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 286, 'total_tokens': 307, 'completion_time': 0.052611655, 'prompt_time': 0.024055591, 'queue_time': 0.084187, 'total_time': 0.076667246}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9504a8bf-fb3a-4208-8262-5f464e8631b3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Egypt'}, 'id': 'm1nn5vt3a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 286, 'output_tokens': 21, 'total_tokens': 307})]}}
{'tool_node': {'messages': [ToolMessage(content='[{"title": "Egypt Weather in September 2025: Temperature Decreasing, No R

In [15]:
input = {
    "messages": ["What is the most common restriction to the agentic ai in 2025?"]
}

events = graph.astream_events(input=input, version="v2")

async for event in events:
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': ['What is the most common restriction to the agentic ai in 2025?']}}, 'name': 'LangGraph', 'tags': [], 'run_id': '5a3eb741-d65d-4207-a919-a53172f581ce', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='What is the most common restriction to the agentic ai in 2025?', additional_kwargs={}, response_metadata={}, id='f1acfa1d-89c5-4262-9013-cffd976a824f')]}}, 'name': 'call_llm', 'tags': ['graph:step:1'], 'run_id': 'be3d9bb1-728e-4a1b-b3cf-53ff26f95c2d', 'metadata': {'langgraph_step': 1, 'langgraph_node': 'call_llm', 'langgraph_triggers': ('branch:to:call_llm',), 'langgraph_path': ('__pregel_pull', 'call_llm'), 'langgraph_checkpoint_ns': 'call_llm:539d6cd0-5139-4ceb-17cd-8a99af1f7fd7'}, 'parent_ids': ['5a3eb741-d65d-4207-a919-a53172f581ce']}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='What is the most common restriction to 

In [16]:
input = {
    "messages": ["What is the most common restriction to the agentic ai in 2025?"]
}

events = graph.astream_events(input=input, version="v2")

async for event in events:
    if event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

The most common restriction to the agentic AI in 2025 is the need for compliance with laws and standards, as well as the potential risks and challenges associated with its development and deployment. This includes ensuring that the AI agent's objectives are aligned with the business strategy and operations, and that compliance measures are in place to mitigate potential risks. Additionally, the use of agentic AI may be restricted by the need to protect user data and prevent unauthorized access or manipulation.